In [ ]:
import jax
import optax
import jax.numpy as jnp

from meta_icl.transformer import Transformer
from meta_icl.data import sample_regression_dataset

In [18]:
# TODO: Manual parameters for now
main_rng = jax.random.key(42)

lr = 3e-1
input_size = 1
num_epochs = 500

In [19]:
# TODO: Improve the transformer ...
model = Transformer(num_heads=1, embed_dim=1, num_layers=1, output_dim=2)

# Initialize parameters
x = jnp.ones((1, 100, 2))
params = model.init(main_rng, x) 
y = model.apply(params, x)

opt_adamw = optax.adamw(learning_rate=lr)
opt_params = opt_adamw.init(params)

In [20]:
def compute_mse_loss(params, model, X, y):
    # TODO: Compare slicing with original codebase
    # print(X.shape, y.shape, y[:, -1].shape)
    y_hat = model.apply(params, X)[:, -1, -1] # inference
    # print(y_hat.shape, y[:, -1].shape)
    # print(y_hat[0], y[0])
    loss = jnp.mean((y_hat - y[:, -1]) ** 2)  # MSE loss
    return loss


for epoch in range(num_epochs):
    X, y, _ = sample_regression_dataset(main_rng, input_size, input_range=1.0)
    
    # TODO: Refactor step to a func with @jax.jit train_step()
    loss_grad_fn = jax.value_and_grad(compute_mse_loss)
    loss, grads = loss_grad_fn(params, model, X, y)

    updates, opt_params = opt_adamw.update(grads, opt_params, params)
    params = optax.apply_updates(params, updates)

    # Created new params, opt_params
    print(loss)

0.08491025
0.13170448
0.07653677
0.08060366
0.08296323
0.07303948
0.089885555
0.07502618
0.078628175
0.084139474
0.074124224
0.07746638
0.0805236
0.07295945
0.076981105
0.07842289
0.073228024
0.07527538
0.077023804
0.07305141
0.074542694
0.075986244
0.07331341
0.07364316
0.07522335
0.0732523
0.0733279
0.074557416
0.073360026
0.07300716
0.07408052
0.07328292
0.07292267
0.07370481
0.07325584
0.07284464
0.073444255
0.073164605
0.07283162
0.07326254
0.073097296
0.07281782
0.07313823
0.07302087
0.07281847
0.07305499
0.0729636
0.07281704
0.072996855
0.072911315
0.07282145
0.07295462
0.07287411
0.07282461
0.07292222
0.072843745
0.07283151
0.0728943
0.07282507
0.07283646
0.07287002
0.07281374
0.072840996
0.07284825
0.07281066
0.07284147
0.07283019
0.07281308
0.07283781
0.07281774
0.072817825
0.072830416
0.07281145
0.072821915
0.07282149
0.07281105
0.07282248
0.07281417
0.07281387
0.07281937
0.07281084
0.07281633
0.072814636
0.07281134
0.072816186
0.0728113
0.07281326
0.07281372
0.07281079
0.07